<a href="https://colab.research.google.com/github/fquinterov/dfp-blender/blob/main/dfp_phase_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
## **Digital Fringe Projection (DFP) Phase Analysis**
---
Code by Fernando Quintero.  11/nov/2021




This notebook contains contains the necessary code to perform a phase analysis of fringe images acquired in a digital fringe projection system.
The method used to extract the phase is by general phase shifting, and then a temporal phase unwrapping by binary encoding is applied to obtain the absolute phase.



##### **Table of Contents**
1.   [Import libraries](#importlib)
2.   [Phase extraction](#pextraction)
3.   [Phase unwrapping](#punwrapping)

#### **1. Import libraries** <a name="importlib"></a>
First we import the `opencv`, `matplotlib`, `numpy`, and `pandas` libraries. If you are not using **Google Colab** make sure you have them installed in your version of Python 3.x for the correct operation of the notebook.

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

#### **2. Phase extraction** <a name="pextraction"></a>
For phase extraction, the phase shifting technique is implemented in its general form. This technique allows to obtain the phase $\phi(x,y)$ of a set of N images of projected sinusoidal fringes, knowing the phase shifts $\varphi _k$ and the intensity $I_k$ in each pixel of the N images by means of the following expression for a k-th shift:

$$
I_k = a + bcos[\phi(x,y) - \varphi ₖ]
$$

The solution of the system is simplified in the following expression to calculate the phase $\phi(x,y)$.:

$$
\phi(x,y) = -arctan(\frac{\sum_{k=1}^{N} I_k sin(\varphi_{k})}{\sum_{k=1}^{N} I_k cos(\varphi_{k})})
$$

The following script shows the example for a phase shifting of 5 fringe images. In the `pitch` variable the periods of the generated fringes must be entered, because the method for phase shifting is by binary coding which will be explained in the next section. Then with a for cycle the images with the phase shifts are read and then the expression to calculate the phase $\phi(x,y)$ is applied.



In [ ]:
pitch=[1280,640,320,160,80,40,20]
plen=np.size(pitch)
wp=np.empty([plen,1024,1280,3])

for i in range(plen):
    i1 = np.double(cv.imread('../input/newtest/' + str(pitch[i]) + '_1.bmp'))
    i2 = np.double(cv.imread('../input/newtest/' + str(pitch[i]) + '_2.bmp'))
    i3 = np.double(cv.imread('../input/newtest/' + str(pitch[i]) + '_3.bmp'))
    i4 = np.double(cv.imread('../input/newtest/' + str(pitch[i]) + '_4.bmp'))
    i5 = np.double(cv.imread('../input/newtest/' + str(pitch[i]) + '_5.bmp'))

    ICN = i1*np.cos(0)+i2*np.cos(2*np.pi/5)+i3*np.cos(4*np.pi/5)+i4*np.cos(6*np.pi/5)+i5*np.cos(8*np.pi/5)
    ISN = i1*np.sin(0)+i2*np.sin(2*np.pi/5)+i3*np.sin(4*np.pi/5)+i4*np.sin(6*np.pi/5)+i5*np.sin(8*np.pi/5)
    phase = -np.arctan2(ISN,ICN)
    
    cv.imwrite('./phase_' + str(pitch[i]) +  '.bmp',phase)
    wp[i] = phase

#### **3. Phase unwrapping** <a name="punwrapping"></a>

Phase shifting technique recover a discontinuous phase φ(x, y), wrapped between (-π, π), with 2π discontinuities. For the 3D analysis it is necessary to obtain a continuous phase, which does not present ambiguities in the values. En este notebook binary coding time-phase unwrapping technique implemented which consists of projecting and capturing fringe patterns with base frequencies of base frequency 2 ($2^k$). The pseudocode for the implementation of the technique is shown below.

> ##### **Algorithm 1: Unwrapping algorithm by binary coding.**
**Input:** Discontinuous phases: $\phi _k$\
**Output:** Absolute Continuous Phase: $\Phi _a$
1.   Initialize $\phi _k$
2.   **for k = 1:K do**
3.   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\Delta_{\phi} = \phi_k  - \phi_{k-1}$
4.   &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\Phi_a = \Phi_a + \Delta_{\phi} + 2\pi$ round$(\frac{\Phi_a - \Delta_{\phi}}{2\pi})$
5.   **end**

And the script of the phase development of the time-temportal phase by geometrical constraints is as follows:

In [ ]:
ap = wp[0]
for k in range(1,plen):
    deltap = wp[k] - wp[k - 1]
    ap = ap + deltap + 2*math.pi*(np.round_((ap - deltap)/(2*math.pi)))